In [ ]:
#Upload the second Dataset from Kaggle
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download impapan/dry-beans-dataset

  0% 0.00/2.45M [00:00<?, ?B/s]
100% 2.45M/2.45M [00:00<00:00, 172MB/s]


In [ ]:
! unzip dry-beans-dataset.zip

Archive:  dry-beans-dataset.zip
replace Dry_Bean_Dataset.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pandas as pd
data2 = pd.read_excel("Dry_Bean_Dataset.xlsx")
data2

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395,610.291,208.178117,173.888747,1.197191,0.549812,28715,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,SEKER
1,28734,638.018,200.524796,182.734419,1.097356,0.411785,29172,191.272750,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430,SEKER
2,29380,624.110,212.826130,175.931143,1.209713,0.562727,29690,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,SEKER
3,30008,645.884,210.557999,182.516516,1.153638,0.498616,30724,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199,SEKER
4,30140,620.134,201.847882,190.279279,1.060798,0.333680,30417,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166,SEKER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13606,42097,759.696,288.721612,185.944705,1.552728,0.765002,42508,231.515799,0.714574,0.990331,0.916603,0.801865,0.006858,0.001749,0.642988,0.998385,DERMASON
13607,42101,757.499,281.576392,190.713136,1.476439,0.735702,42494,231.526798,0.799943,0.990752,0.922015,0.822252,0.006688,0.001886,0.676099,0.998219,DERMASON
13608,42139,759.321,281.539928,191.187979,1.472582,0.734065,42569,231.631261,0.729932,0.989899,0.918424,0.822730,0.006681,0.001888,0.676884,0.996767,DERMASON
13609,42147,763.779,283.382636,190.275731,1.489326,0.741055,42667,231.653248,0.705389,0.987813,0.907906,0.817457,0.006724,0.001852,0.668237,0.995222,DERMASON


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import linalg as la
classifications = data2.to_numpy()[:,-1]
dry_beans = data2.to_numpy()[:,:-1]
dry_beans = dry_beans.T
max_vals = np.amax(dry_beans, axis=1)
print(max_vals)
dry_beans = dry_beans / max_vals[:,None]
print(dry_beans)
classifications = data2.to_numpy()[:,-1]
beans = {}
for i in classifications:
  if i not in beans:
    beans[i] = 1
  else:
    beans[i] = beans[i] + 1
print(beans)
size = len(beans)
print(size)
plt.scatter(dry_beans[3], dry_beans[13], marker = '.', color = 'b')

In [ ]:
#data2
import matplotlib.pyplot as plt
from numpy import linalg as la
classifications = data2.to_numpy()[:,-1]
mean = np.mean(dry_beans, axis = 1) #mean of the data
mean = mean[:, None]
A = dry_beans-mean
cov = A@A.T
cov = np.array(cov, dtype=float)
lambda_old, v_old = la.eig(cov)
def eigsort(V, eigvals):
  lohival = np.sort(eigvals)
  lohiindex = np.argsort(eigvals)
  lambd = np.flip(lohival)
  index = np.flip(lohiindex)
  Dsort = np.diag(lambd)
  M = np.size(lambd)
  Vsort = np.zeros((M, M))
  for i in range(M):
    Vsort[:, i] = V[:,index[i]]
  return Vsort, Dsort
v_new, lambda_new = eigsort(v_old, lambda_old)
pca_data = v_new.T@(A)
#plotting the first two dimensions of the PCA processed data
plt.scatter(pca_data[0], pca_data[1], marker = '.', color = 'b')

In [ ]:
import time
#This implementation is used as a sanity check for the Kmean algorithm. To generate the results shown in the report,
#we used Alexis's implementation.
def calcSqDistances(X, Kmus, D):
    dis = np.zeros((len(X), len(Kmus)))
    for k in range(0, len(Kmus)):
        for i in range(0, len(X)):
          for x in range(0, D):
            dis[i][k] = dis[i][k] + (Kmus[k][x] - X[i][x])**2
    print('sum', sum(sum(dis)))
    return dis

def determineRnk(sqDmat):
    belong = [0]*len(sqDmat)
    for i in range(0, len(sqDmat)):
        element = sqDmat[i, :].tolist()
        belong[i] = element.index(min(element))
    return belong

def recalcMus(X, Rnk, k, D):
    centers = {}
    counter = {}
    for i in range(0, len(X)):
      if Rnk[i] not in centers:
        centers[Rnk[i]] = X[i,:]
        counter[Rnk[i]] = 1
      else:
        centers[Rnk[i]] = centers[Rnk[i]] + X[i,:]
        counter[Rnk[i]] = counter[Rnk[i]] + 1
    for key in centers:
      centers[key] = centers[key]/counter[key]
    return np.array(list(centers.values()))

def runKMeans(K, data):
    X = data
    N, D = X.shape
    Kmus = np.zeros((K, D))
    rand_inds = np.random.permutation(N)
    print(rand_inds)
    Kmus = X[rand_inds[0:K],:]

    #specify the maximum number of iterations to allow
    maxiters = 1000

    for iter in range(maxiters):
        sqDmat = calcSqDistances(X, Kmus, D)
        #print(len(sqDmat))
        Rnk = determineRnk(sqDmat)
        KmusOld = Kmus
        time.sleep(1)
        Kmus = recalcMus(X, Rnk, K, D)
        #print(Kmus)
        if np.sum(np.abs(KmusOld.reshape((-1, 1)) - Kmus.reshape((-1, 1)))) < 1e-6:
            print(iter)
            break
    print(Kmus)
    return Kmus

In [ ]:
#Kmean clustering on the original dataset
inputs = dry_beans.T
start_time = time.time()
result = runKMeans(size, inputs)
print('runtime', time.time() - start_time)

(13611, 16)
distance 70.1416735367053
distance 70.83781852801167
distance 70.91111014202204
distance 70.92947377598394
distance 70.94196058838556
distance 70.94859838454691
distance 70.95070647748052
distance 70.95118213512534
distance 70.95147071769762
distance 70.9518013467472
distance 70.95181706770644
distance 70.95113974738221
distance 70.94995461797454
distance 70.94935128023636
distance 70.94881556529972
distance 70.94825202549549
distance 70.94749215233287
distance 70.94729224710015
distance 70.94708512609647
distance 70.94684800370287
distance 70.94675841462373
distance 70.94673903293086
distance 70.94663688022688
distance 70.94663494059836
distance 70.94663494059836
24
runtime 3.2875607013702393


In [ ]:
#Kmean clustering on the PCA processed dataset 
start_time = time.time()
result = runKMeans(size, pca_data[0:2,:].T)
print('runtime in second', time.time() - start_time)
print('result', result)

(13611, 2)
distance -0.05050025052267515
distance -0.14489459940658578
distance -0.3734143988325806
distance -0.8103114624740333
distance -1.2030304669455312
distance -1.3317180290558799
distance -1.4088900009877086
distance -1.4558534210995018
distance -1.4722562264813035
distance -1.4743472387556589
distance -1.4723184438524788
distance -1.47055165208841
distance -1.4709908632000896
distance -1.4745861736189896
distance -1.4771085087973734
distance -1.4775530299315873
distance -1.4777302127924563
distance -1.4790634889038095
distance -1.4796775042100796
distance -1.4801521500182768
distance -1.4806961645862788
distance -1.4805860082151023
distance -1.480654816657257
distance -1.4807249673534582
distance -1.4807249673534582
24
runtime in second 0.71639084815979
result (array([[-0.18505452019130036, -0.11730082633037225],
       [0.33833478450531373, -0.2604819211835939],
       [-0.34923115212607536, 0.004978403570461329],
       [-0.9435125501230854, -0.48421719136493224],
       [0.